In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:



# Example usage:

In [ ]:

def create_rectangle(init_point = [0,0], height = 10,width = 10):
    x0 = init_point
    x1 = [init_point[0],init_point[1]+height]
    x2 = [init_point[0]+width, init_point[1]+height]
    x3 = [init_point[0]+width, init_point[1]]
    return [x0,x1,x2,x3]


def safe_zone_coords(image_shape):
    height = image_shape[1][1] - image_shape[0][1]
    width = image_shape[3][0] - image_shape[0][0]
    top = height*5/100
    bottom = height*15/100
    left = width*5/100
    right = width*10/100
    init_point = image_shape[0]
    init_point = [init_point[0]+left, init_point[1]+bottom]
    new_height = height-top-bottom
    new_width = width-left-right
    return create_rectangle(init_point, new_height, new_width)
def plot_boxes(*args):
    """
    Plots multiple boxes given coordinates of their vertices.

    Parameters:
        *args (list of lists): Variable number of lists containing coordinates of the box vertices.

    Returns:
        None
    """
    # Create a plot
    fig, ax = plt.subplots()

    for coordinates in args:
        # Extract x and y coordinates from the input list
        x_coords = [point[0] for point in coordinates]
        y_coords = [point[1] for point in coordinates]

        # Append the first point to the end to close the box
        x_coords.append(x_coords[0])
        y_coords.append(y_coords[0])

        # Plot the box
        ax.plot(x_coords, y_coords, color='b')  # Plot the box edges
        ax.fill(x_coords, y_coords, color='b', alpha=0.3)  # Fill the box

        # Add coordinate labels
        for (x, y) in zip(x_coords[:-1], y_coords[:-1]):
            ax.text(x, y, f'({x}, {y})', fontsize=8, ha='right')

    # Set labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Box Plot')

    # Display the plot
    plt.grid(True)
    plt.axis('equal')  # Set equal aspect ratio
    plt.show()
image_shape = create_rectangle([0,0], 120,100)
rec_box = create_rectangle([20,20],20,20)
plot_boxes(rec_box, image_shape, safe_zone_coords(image_shape))
safe_zone = safe_zone_coords(image_shape)

In [ ]:
def find_area(coords):
    """
    Calculates the area of a polygon given its coordinates using the Shoelace formula.

    Parameters:
        coords (list of lists): Coordinates of the vertices of the polygon.

    Returns:
        float: Area of the polygon.
    """
    n = len(coords)
    area = 0
    for i in range(n):
        j = (i + 1) % n
        area += coords[i][0] * coords[j][1]
        area -= coords[j][0] * coords[i][1]
    area = abs(area) / 2.0
    return area

big_box = safe_zone
little_boxes = [create_rectangle([20,20],40,30), create_rectangle([20,20],20,20)]
box_num = 0
box = little_boxes[box_num]

def get_max_area_coords(big_box, little_box):    
    ys_box = list(set(i[1] for i in box))
    xs_box = list(set([i[0] for i in box]))
    xs_box.sort()

    xs_bb = list(set([i[0] for i in big_box]))
    ys_bb = list(set([i[1] for i in big_box]))

    ys = ys_bb+ys_box; ys.sort()

    area = 0
    for i in range(len(ys)-1):
        area_box = [xs_bb[0],ys[i]], [xs_bb[0],ys[i+1]], [xs_bb[1],ys[i+1]], [xs_bb[1],ys[i]]
        bb = list(set([i[1] for i in area_box]))
        bb.sort()
        if bb != xs_box:
            temp_area = find_area(area_box)
            if temp_area>area:
                area = temp_area
                coords = area_box
        if bb==xs_box:
            
            
        
    return area, coords



for box in little_boxes:
    area, coords = get_max_area_coords(big_box,box)
    plot_boxes(image_shape, coords, box )

In [ ]:
def find_diff(p1,p2):
    h = abs(p2[1]-p1[1])
    w = abs(p2[0]-p1[0])
    return h*w
    
    
safe_zone = safe_zone_coords(image_shape)
box = rec_box

area = 0

for i in safe_zone:
    for j in box:
        temp_area = find_diff(i,j)
        if temp_area>area:
            coords = [i,j]
            
print(coords)    

In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
BTC_Ticker = yf.Ticker("BTC-USD")
BTC_Data = BTC_Ticker.history(period="max")

In [ ]:
ticker = yf.Ticker('NMT-USD')
data = ticker.history(period='max',interval = '1m')
data.head()

In [ ]:
data['PercentageChange'] = round(data.Open.pct_change()*100,3)
data

In [ ]:
data['Close'] = data.Open.shift(-1)
data['direction'] = data['Close'] - data['Open'] >=0
data

In [ ]:
\

In [ ]:
data['Date'] = data.index.strftime('%Y-%m-%d')
highs = data.groupby('Date')[['High']].max()
lows = data.groupby('Date')[['Low']].min()
df =highs.merge(lows, on = 'Date')
df['Change'] = (df['High'] - df['Low'])*100/df['Low']
df

In [ ]:
!pip install requests_html

In [22]:
import requests
# from requests_html import HTMLSession
# session = HTMLSession()
from bs4 import BeautifulSoup
num_currencies=250
resp = requests.get(f"https://finance.yahoo.com/crypto?offset=0&count={num_currencies}")


In [ ]:

tables = pd.read_html(resp.text)
df = tables[0].copy()
symbols_yf = df.Symbol.tolist()
symbols_yf

In [33]:
symbols = df.Symbol.unique()
final_df = pd.DataFrame()
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    data = ticker.history(start = '2024-05-01', end= '2024-08-01', interval = '1d')
    data['Symbol'] = symbol
    final_df = pd.concat([final_df, data])


In [ ]:
df = final_df.reset_index()[['Date','Symbol','Open']]
df = df.pivot_table(columns = ['Symbol'], index = 'Date')
df.columns = df.columns.get_level_values(1)
df.corr()[['BTC-USD']].sort_values('BTC-USD', ascending = False).head(10)

In [ ]:
import numpy as np
df = final_df[['Symbol','Open']]
df
change = pd.DataFrame()
for symbol in df.Symbol.unique():
    temp = df[df.Symbol == symbol].drop(columns = 'Symbol')
    temp['PctCh'] = round(temp['Open'].pct_change()*100,2)
    temp['Symbol'] = symbol
    change = pd.concat([change, temp])
change = change.reset_index().drop(columns = 'Open').pivot_table(columns = 'Symbol', index = 'Date').dropna()
change.columns = change.columns.get_level_values(1)
diff = pd.DataFrame()
for symbol in change.columns.unique():
    temp = change[[symbol,'BTC-USD']]
    rmse = np.sqrt(np.mean((temp['BTC-USD'] - temp[symbol])**2))
    
    difference = pd.DataFrame({'Symbol':[symbol],
                        'RMSE':[rmse]})
    diff = pd.concat([diff, difference])
    
change = change.corr()[['BTC-USD']].sort_values('BTC-USD', ascending = False).dropna().reset_index().merge(diff, on = ['Symbol'])
change


introduce a lag into the temp dfs to see which coins jump right after btc jumps instead of looking at same timeframe correlation. Look at a leading correlatioon and rmse to see which alt coins will jump next


In [ ]:
# df['Change'] = df['% Change'].str.replace('%','').str.replace('+','').astype(float)
df[df['Price (Intraday)']<0.09]
small = df[df['Market Cap'].str.contains('M')]
# Define a mapping dictionary for suffixes
def convert_to_scientific(value):
    suffixes = {'B': 1e9, 'M': 1e6,'T':1e12}
    if value[-1] in suffixes:
        return float(value[:-1]) * suffixes[value[-1]]
    else:
        return float(value)

# Apply the function to the 'Circulating Supply' column and assign it to 'Supply'
small['Supply'] = small['Circulating Supply'].apply(convert_to_scientific)
small['ExpectedPrice'] = 1e9/small['Supply']
small = small[small['Price (Intraday)']<small['ExpectedPrice']].head(14)
small['Expected Increase'] = round(small['ExpectedPrice']*100/small['Price (Intraday)'],2)
small[['Symbol','Name','Price (Intraday)','ExpectedPrice','Expected Increase', '% Change']]

In [4]:
df['Supply'] = df['Circulating Supply'].apply(convert_to_scientific)
df['Market Cap'] = df['Market Cap'].apply(convert_to_scientific)
df =df[['Symbol','Name', 'Price (Intraday)','Market Cap','Supply']]


In [ ]:
df['Expected'] = 1e9/df['Supply']
df

In [2]:
def check_perc_change(x1, x2):
    inc = (x2-x1)*100/x1
    return inc


In [80]:
pendle = yf.Ticker('IOTX-USD')
pendle = pendle.history(period = 'max')

In [ ]:
pendle[pendle.index >= '2024-01-01'][['Open','High','Low']].plot()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()
pendle = pendle[pendle.index>'2024-01-01']
pendle

In [25]:
from statsmodels.tsa.tsatools import detrend



In [ ]:
from math import pi
detrended = detrend(pendle.Open, order=2)
detrended = pd.DataFrame(detrended)

x = 0.03
detrended = detrended.reset_index().drop(columns = 'Date')
detrended['pi'] = np.linspace(0,2*pi, len(detrended))
detrended.set_index('pi', inplace = True)
detrended['Sin'] = max(detrended.Open)*np.sin(np.linspace(0, 2*np.pi*10, len(detrended)))
detrended.plot(figsize = (15,5), marker = 'o')



In [ ]:
trend = pendle.Open - detrended.values.flatten()  # Extracting the trend from the original series
detrended_with_trend = pd.DataFrame(detrended.values + trend.values.reshape(-1, 1), index=pendle.index, columns=['Open'])


In [ ]:
plt.plot([i for i in range(100)], np.sin(np.linspace(0, 2*np.pi, 100)))

In [ ]:
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)
plt.plot(x,y)